# 迭代

罗马不是一天建成的。同样，神经网络模型也不是一次训练就可以学会的。

在深度学习中，模型训练遵循一个朴素的规则：利用海量数据，通过成千上万次的**反复试错**，以**小步快跑**的方式逐渐逼近最佳模型参数。这种**重复且不断改进**的过程，称为**迭代**（Iteration）。

In [1]:
import numpy as np

## 数据集

模型训练所需要的海量数据，称为**数据集**（Dataset）。

数据集的每一条数据，称为一个**样本**（Sample）。每个样本都需要包括**特征值**和**标签值**两部分。

---

通常数据集会被随机分成两部分：
* **训练集**（Training Set）：较多的一部分（比如：80%），用于模型训练；
* **测试集**（Test Set）：另一部分较少的（比如：20%），用于模型评估。

划分数据集的目的是为了保留一部分**新数据**：测试集。这样，可以用**新数据**验证训练过的网络模型是否真正学会了规律，还是只死记硬背了一些内容。

这就像是期末考试，老师不会用你做过的练习题做考卷，而是用一套新题来考察你是不是真的掌握了学习的知识。

### 训练数据：特征、标签

小明提供了过去四天的天气预报和他的销售记录。

我们把这些数据保存在 NumPy 的二维数组中，构成了我们的训练集。

In [2]:
train_features = np.array([[22.5, 72.0],
                           [31.4, 45.0],
                           [19.8, 85.0],
                           [27.6, 63.0]])
train_labels = np.array([[95.0],
                        [210.0],
                        [70.0],
                        [155.0]])

### 测试数据：特征、标签

我们之前使用的数据则作为测试集：

In [3]:
test_features = np.array([[28.1, 58.0]])
test_labels = np.array([[165.0]])

<div style="border-left: 4px solid #4CAF50; background:#f9f9f9; padding:10px; margin:10px 0;">
<strong>💡 提示：</strong> 数据集中的数据，都以<strong>二维数组</strong>的形式保存。以后，我们还会遇到更多维度数组保存的数据。
</div>

## 模型

### 参数：权重、偏置

In [4]:
weight = np.ones(2) / 2
bias = np.zeros(1)

### 推理函数

In [5]:
def forward(x, w, b):
    return x @ w.T + b

### 损失函数（均方误差）

In [6]:
def mse_loss(p, y):
    return np.mean(np.square(y - p))

### 梯度函数

In [7]:
def gradient(p, y):
    return - 2 * (y - p)

### 反向函数

In [8]:
def backward(x, d, w, b, lr):
    w = w - d * x * lr
    b = b - d * lr
    return w, b

## 训练

### 超参数：学习率

In [9]:
LEARNING_RATE = 0.00001

### 迭代

我们把一个样本提供给网络模型学习，称为**一次迭代**。迭代包括三个步骤：

1. **前向传播**

```{figure} images/forward.png
:align: center
:width: 300px
```

* $x$：特征值；
* $w, b$：权重和偏置；
* $p$：预测值。

2. **梯度计算**

```{figure} images/gradient.png
:align: center
:width: 480px
```

* $p$：预测值；
* $y$：标签值；
* $l$：损失值；
* $d$：误差项。

3. **反向传播**

```{figure} images/backward.png
:align: center
:width: 300px
```

* $d$：误差项；
* $x$：特征值；
* $w, b$：权重和偏置。

<div style="border-left: 4px solid #4CAF50; background:#f9f9f9; padding:10px; margin:10px 0;">
<strong>💡 定义：</strong> 每完成一次参数更新，称为一次迭代。
</div>

顺序或者随机地把训练集中所有样本都迭代一次，称为**一轮迭代**。

我们将训练集中的 4 个样本依次用于训练，共完成 4 次迭代。

In [10]:
for i in range(len(train_features)):
    # 依次读入训练数据：特征、标签
    feature = train_features[i]
    label = train_labels[i]

    # 前向传播
    prediction = forward(feature, weight, bias)
    # 梯度计算
    delta = gradient(prediction, label)
    # 反向传播
    weight, bias = backward(feature, delta, weight, bias, LEARNING_RATE)

print(f"weight: {weight}")
print(f"bias: {bias}")

weight: [0.67678017 0.8307117 ]
bias: [0.0060984]


可以看到，经过一轮迭代，权重和偏置都被合理地更新了。那么效果会怎样？

## 验证

模型训练完成以后，要用不同的数据（测试集）来评估模型训练的效果。

### 推理

In [11]:
predictions = forward(test_features, weight, bias)
print(f'predictions: {predictions}')

predictions: [67.20489976]


### 评估

In [12]:
loss = mse_loss(predictions, test_labels)
print(f'loss: {loss}')

loss: 9563.881631333801


从验证结果来看，经过一轮迭代，损失值继续下降，网络模型的效果又有提高。但是仍然有很大的提升空间。